# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [3]:
from autogen import ConversableAgent

# human_input_mode enables HITL(Human in the loop capability)

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER", # Never ask for human input.
)

 We can switch to different human_input_modes like 'ALWAYS' ,where it will always ask for human input before trying to generate its own reply. 

In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure thing! Here's a joke for you:

Why did the coffee file a police report?
It got mugged!


In [6]:
# This doesn't maintain state or conversation history and 
# generates reply for a standalone input

reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Of course! Please tell me the joke you'd like me to repeat for you.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

Do define system message to establish & customize the agent behaviour, otherwise it will have an empty system message and will perform like a general purpose assistant agent

In [7]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [8]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, Cathy! Looks like that scarecrow took home the "Crop-tastic Award"!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I guess you could say he really knocked 'em dead in the competition!

--------------------------------------------------------------------------------


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [9]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': 'Hey Joe! Why did the scarecrow win an award? Because he was '
             'outstanding in his field!',
  'role': 'user'},
 {'content': 'Haha, Cathy! Looks like that scarecrow took home the '
             '"Crop-tastic Award"!',
  'role': 'assistant'},
 {'content': "Haha, I guess you could say he really knocked 'em dead in the "
             'competition!',
  'role': 'user'}]


In [10]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 61,
                                                             'cost': 0.00018800000000000002,
                                                             'prompt_tokens': 193,
                                                             'total_tokens': 254},
                                      'total_cost': 0.00018800000000000002},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 61,
                                                             'cost': 0.00018800000000000002,
                                                             'prompt_tokens': 193,
                                                             'total_tokens': 254},
                                      'total_cost': 0.00018800000000000002}}


In [11]:
pprint.pprint(chat_result.summary)

"Haha, I guess you could say he really knocked 'em dead in the competition!"


## Get a better summary of the conversation

In [12]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, Cathy! Looks like that scarecrow took home the "Crop-tastic Award"!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I guess you could say he really knocked 'em dead in the competition!

--------------------------------------------------------------------------------


In [13]:
pprint.pprint(chat_result.summary)

('The conversation was lighthearted and centered around jokes about a '
 'scarecrow winning awards in a competition.')


With summary_method = ‘reflect_with_llm’, an LLM reflects on the summary and produces a new summary.


## Chat Termination

Chat can be terminated using a termination conditions, instead of defining a pre-defined no. of turns.

In [14]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [15]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, glad you're enjoying the jokes! Did you hear about the mathematician who's afraid of negative numbers? He'll stop at nothing to avoid them!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! I wonder if he has imaginary friends too!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, maybe he does! I bet they have complex conversations together!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, Cathy, you're on fire with these math jokes! Who knew numbers could be so much fun?

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! Numbers can be surprisingly entertaining, right

Conversation state is still retained and used when another agent resumes the chat with `send()`

In [16]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was the mathematician who's afraid of negative numbers and will stop at nothing to avoid them!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, thanks for the reminder! It was great chatting with you, Joe. Take care and keep laughing! If you ever need more jokes, you know where to find me. I gotta go.

--------------------------------------------------------------------------------


#### Reference 
* https://microsoft.github.io/autogen/0.2/docs/tutorial/introduction